In [4]:
from fmriprep import config
import os
import nibabel as nb
from nipype.interfaces.fsl import Split as FSLSplit
from nipype.pipeline import engine as pe
from nipype.interfaces import utility as niu
from niworkflows.utils.connections import pop_file, listify
from fmriprep.interfaces import DerivativesDataSink
from fmriprep.interfaces.reports import FunctionalSummary

# BOLD workflows
from fmriprep.workflows.bold.confounds import init_bold_confs_wf, init_carpetplot_wf
from fmriprep.workflows.bold.hmc import init_bold_hmc_wf
from fmriprep.workflows.bold.registration import init_bold_t1_trans_wf, init_bold_reg_wf
from fmriprep.workflows.bold.resampling import (
    init_bold_std_trans_wf,
    init_bold_preproc_trans_wf,
)
# from fmriprep.workflows.bold.outputs import init_func_derivatives_wf
from bids import BIDSLayout
import numpy as np
from fmriprep.config import DEFAULT_MEMORY_MIN_GB
import os
from linescanning import transform
from linescanning.fmriprep import bold_confs_wf 

In [57]:
fprep_dir = "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/derivatives/fmriprep"
bold_file = "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/derivatives/fmriprep/sub-008/ses-2/func/sub-008_ses-2_task-SRFi_acq-3DEPI_run-1_desc-preproc_bold.nii.gz"
bold_mask = "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/derivatives/fmriprep/sub-008/ses-2/func/sub-008_ses-2_task-SRFi_acq-3DEPI_run-1_desc-brain_mask.nii.gz"
movpar_file = "/data1/projects/MicroFunc/Jurjen/programs/packages/fmriprep/AgeRF_tdm/fmriprep_wf/single_subject_03_wf/func_preproc_ses_1_task_scenes_run_0_acq_3DEPI_wf/bold_hmc_wf/normalize_motion/motion_params.txt"
rmsd_file = "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/bold_hmc_wf/mcflirt/sub-008_ses-2_task-SRFi_run-1_acq-3DEPI_bold_valid_mcf.nii.gz_rel.rms"
t1_mask = "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/derivatives/manual_masks/sub-008/ses-1/sub-008_ses-1_acq-MP2RAGE_desc-brainmask.nii.gz"
t1_tmps = [
    "/data1/projects/MicroFunc/Jurjen/projects/VE-pRF/derivatives/manual_masks/sub-008/ses-1/sub-008_ses-1_acq-MP2RAGE_label-GM_probseg.nii.gz",
    "/data1/projects/MicroFunc/Jurjen/projects/VE-pRF/derivatives/manual_masks/sub-008/ses-1/sub-008_ses-1_acq-MP2RAGE_label-WM_probseg.nii.gz",
    "/data1/projects/MicroFunc/Jurjen/projects/VE-pRF/derivatives/manual_masks/sub-008/ses-1/sub-008_ses-1_acq-MP2RAGE_label-CSF_probseg.nii.gz"]
workdir = "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf"
bids_dir = "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/derivatives/fmriprep"
t1_bold_xform = "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/derivatives/pycortex/sub-008/transforms/sub-008_from-ses1_to-ses2_rec-motion1_desc-genaff.mat"
apply_warp = False

regressors_all_comps = False
regressors_dvars_th = 1.5_mask
        self.bold_confounds_wf.inputs.inputnode.movpar_file     = self.movpar_file
        self.bold_confounds_wf.inputs.inputnode.t1w_mask        = self.t1_mask
        self.bold_confounds_wf.inputs.inputnode.t1w_tpms        = self.t1_tmps
        self.bold_confounds_wf.inputs.inputnode.t1_bold_xform   = self.t1_bold_xform
        self.bold_confounds_wf.inputs.inputnode.rmsd_file       = self.rm
regressors_fd_th = 0.5

In [10]:
# get bids layout
layout = BIDSLayout(bids_dir, validate=False)

# Extract metadata
all_metadata = [layout.get_metadata(fname) for fname in listify(bold_file)]

# Take first file as reference
boldref = pop_file(bold_file)
metadata = all_metadata[0]

if os.path.isfile(boldref):
    bold_tlen, mem_gb = bold_confs_wf._create_mem_gb(boldref)

In [62]:
# get confounds
bold_confounds_wf = init_bold_confs_wf(
    mem_gb=mem_gb,
    metadata=metadata,
    regressors_all_comps=regressors_all_comps,
    regressors_fd_th=regressors_dvars_th,
    regressors_dvars_th=regressors_fd_th,
    name="bold_confounds_wf",
)

# this is necessary for the reports!
bold_confounds_wf.get_node("inputnode").inputs.t1_transform_flags = [False]
for node in bold_confounds_wf.list_node_names():
    if node.split(".")[-1].startswith("ds_report"):
        bold_confounds_wf.get_node(node).inputs.base_directory = fprep_dir
        bold_confounds_wf.get_node(node).inputs.source_file = boldref

# apply T1w > BOLD transformation to T1w mask
if apply_warp:
    bold_mask = transform.ants_applytrafo(
        bold_file, 
        t1_mask, 
        trafo=t1_bold_xform, 
        invert=0, 
        interp='mul', 
        output=bold_mask, 
        return_type="file", 
        verbose=False)

In [63]:
# set inputs
bold_confounds_wf.inputs.inputnode.bold = bold_file
bold_confounds_wf.inputs.inputnode.bold_mask = bold_mask
bold_confounds_wf.inputs.inputnode.movpar_file = movpar_file
bold_confounds_wf.inputs.inputnode.t1w_mask = t1_mask
bold_confounds_wf.inputs.inputnode.t1w_tpms = t1_tmps
bold_confounds_wf.inputs.inputnode.t1_bold_xform = t1_bold_xform
bold_confounds_wf.inputs.inputnode.rmsd_file = rmsd_file

# set workdir
bold_confounds_wf.base_dir = workdir

In [109]:
res = bold_confounds_wf.run()

230608-12:46:00,523 nipype.workflow INFO:
	 Workflow bold_confounds_wf settings: ['check', 'execution', 'logging', 'monitoring']
230608-12:46:00,633 nipype.workflow INFO:
	 Running serially.
230608-12:46:00,634 nipype.workflow INFO:
	 [Node] Setting-up "bold_confounds_wf.dvars" in "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/bold_confounds_wf/dvars".
230608-12:46:00,638 nipype.workflow INFO:
	 [Node] Cached "bold_confounds_wf.dvars" - collecting precomputed outputs
230608-12:46:00,639 nipype.workflow INFO:
	 [Node] "bold_confounds_wf.dvars" found cached.
230608-12:46:00,639 nipype.workflow INFO:
	 [Node] Setting-up "bold_confounds_wf.fdisp" in "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/bold_confounds_wf/fdisp".
230608-12:46:00,642 nipype.workflow INFO:
	 [Node] Cached "bold_confounds_wf.

In [205]:
def init_func_derivatives_wf(
    bids_dir=None,
    workdir=None
    ):

    from fmriprep.interfaces import DerivativesDataSink
    from nipype.pipeline import engine as pe
    from niworkflows.engine.workflows import LiterateWorkflow as Workflow
    from nipype.interfaces import utility as niu
    from smriprep.workflows.outputs import _bids_relative

    name='func_derivatives_wf'
    workflow = Workflow(name=name)

    inputnode = pe.Node(
        niu.IdentityInterface(
            fields=[
                'source_file',
                'all_source_files',
                'confounds',
                'confounds_metadata',
                'anat2bold_xfm',
                'acompcor_masks',
                'tcompcor_mask',
            ]
        ),
        name='inputnode',
    )

    raw_sources = pe.Node(niu.Function(function=_bids_relative), name='raw_sources')
    raw_sources.inputs.bids_root = fprep_dir

    ds_confounds = pe.Node(
        DerivativesDataSink(
            base_directory=fprep_dir,
            desc='confounds',
            suffix='timeseries',
            dismiss_entities=("echo",),
        ),
        name="ds_confounds",
        run_without_submitting=True,
        mem_gb=config.DEFAULT_MEMORY_MIN_GB,
    )
    ds_ref_t1w_xfm = pe.Node(
        DerivativesDataSink(
            base_directory=fprep_dir,
            to='T1w',
            mode='image',
            suffix='xfm',
            extension='.txt',
            dismiss_entities=('echo',),
            **{'from': 'scanner'},
        ),
        name='ds_ref_t1w_xfm',
        run_without_submitting=True,
    )
    ds_ref_t1w_inv_xfm = pe.Node(
        DerivativesDataSink(
            base_directory=fprep_dir,
            to='scanner',
            mode='image',
            suffix='xfm',
            extension='.txt',
            dismiss_entities=('echo',),
            **{'from': 'T1w'},
        ),
        name='ds_t1w_tpl_inv_xfm',
        run_without_submitting=True,
    )

    workflow.connect([
        (inputnode, raw_sources, [('all_source_files', 'in_files')]),
        (inputnode, ds_confounds, [('source_file', 'source_file'),
                                    ('confounds', 'in_file'),
                                    ('confounds_metadata', 'meta_dict')]),
        (inputnode, ds_ref_t1w_inv_xfm, [('source_file', 'source_file'),
                                            ('anat2bold_xfm', 'in_file')]),
    ])

    return workflow

In [206]:
def fetch_output(wf, node, key, graph=None):

    # convert nodes in graph to strings 
    str_nodes = [str(ii) for ii in list(graph.nodes)]

    # then look for index in string-version of nodes
    nn = f'{wf}.{node}'
    ix_rel = str_nodes.index(nn)

    # and fetch the node from the original graph
    try:
        out = list(res.nodes)[ix_rel].result.outputs.get()[key]
    except:
        out = list(res.nodes)[ix_rel].result.outputs
    
    return out


In [214]:
fetch_output("bold_confounds_wf", "acc_msk_bin", "out_file", graph=res)

Bunch(out_file=['/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/bold_confounds_wf/acc_msk_bin/mapflow/_acc_msk_bin0/sub-008_ses-1_acq-MP2RAGE_label-CSF_probseg_trans_masked_masked.nii.gz',
       '/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/bold_confounds_wf/acc_msk_bin/mapflow/_acc_msk_bin1/acompcor_wm_trans_masked_masked.nii.gz',
       '/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/bold_confounds_wf/acc_msk_bin/mapflow/_acc_msk_bin2/acompcor_wmcsf_trans_masked_masked.nii.gz'],
      out_mask=<undefined>)

In [219]:
func_derivatives_wf = init_func_derivatives_wf(bids_dir=fprep_dir)

func_derivatives_wf.inputs.inputnode.all_source_files = bold_file
func_derivatives_wf.inputs.inputnode.source_file = bold_file
func_derivatives_wf.inputs.inputnode.confounds = fetch_output("bold_confounds_wf", "spike_regressors", "confounds_file", graph=res)
func_derivatives_wf.inputs.inputnode.confounds_metadata = fetch_output("bold_confounds_wf", "merge_confound_metadata2", "out_dict", graph=res)
func_derivatives_wf.inputs.inputnode.anat2bold_xfm = t1_bold_xform
func_derivatives_wf.inputs.inputnode.tcompcor_mask = fetch_output("bold_confounds_wf", "tcompcor", "high_variance_masks", graph=res)
func_derivatives_wf.inputs.inputnode.acompcor_masks = fetch_output("bold_confounds_wf", "acc_msk_bin", "out_file", graph=res)
func_derivatives_wf.base_dir = workdir

In [220]:
func_derivatives_wf.inputs.inputnode


acompcor_masks = Bunch(out_file=['/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/bold_confounds_wf/acc_msk_bin/mapflow/_acc_msk_bin0/sub-008_ses-1_acq-MP2RAGE_label-CSF_probseg_trans_masked_masked.nii.gz', '/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/bold_confounds_wf/acc_msk_bin/mapflow/_acc_msk_bin1/acompcor_wm_trans_masked_masked.nii.gz', '/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/bold_confounds_wf/acc_msk_bin/mapflow/_acc_msk_bin2/acompcor_wmcsf_trans_masked_masked.nii.gz'], out_mask=<undefined>)
all_source_files = /data1/projects/MicroFunc/Jurjen/projects/VE-SRF/derivatives/fmriprep/sub-008/ses-2/func/sub-008_ses-2_task-SRFi_acq-3DEPI_run-1_desc-preproc_bold.nii.gz
anat2bold_xfm = /data

In [221]:
func_derivatives_wf.run()

230608-13:24:39,447 nipype.workflow INFO:
	 Workflow func_derivatives_wf settings: ['check', 'execution', 'logging', 'monitoring']
230608-13:24:39,476 nipype.workflow INFO:
	 Running serially.
230608-13:24:39,477 nipype.workflow INFO:
	 [Node] Setting-up "func_derivatives_wf.raw_sources" in "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/func_preproc_ses_2_task_SRFi_run_1_acq_3DEPI_wf/func_derivatives_wf/raw_sources".
230608-13:24:39,480 nipype.workflow INFO:
	 [Node] Outdated cache found for "func_derivatives_wf.raw_sources".
230608-13:24:39,492 nipype.workflow INFO:
	 [Node] Executing "raw_sources" <nipype.interfaces.utility.wrappers.Function>
230608-13:24:39,495 nipype.workflow INFO:
	 [Node] Finished "raw_sources", elapsed time 0.000606s.
230608-13:24:39,505 nipype.workflow INFO:
	 [Node] Setting-up "func_derivatives_wf.ds_confounds" in "/data1/projects/MicroFunc/Jurjen/projects/VE-SRF/sourcedata/sub-008/ses-2/single_subject_008_wf/f